In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from scipy.stats import norm

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind
from scipy.stats import chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
#  MÉTODOS DE VISUALIZACIÓN QUE NO CONOCIAMOS:

# establece el estilo del fondo de las gráficas como "whitegrid". Las gráficas tendrán un fondo blanco con líneas de cuadrícula horizontales y verticales.
sns.set_style("whitegrid")

# cambiará el esquema de colores predeterminado de Seaborn a uno más oscuro, adecuado para gráficas con fondos claros. 
sns.set_theme(style="dark")

# establece el estilo en Solarize_Lihgt2. 
plt.style.use('Solarize_Light2')

In [3]:
# Lectura Fichero 
df = pd.read_csv("marketing_AB.csv", sep =",")
df.head()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14


In [4]:
df.tail()

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
588096,588096,1278437,ad,False,1,Tuesday,23
588097,588097,1327975,ad,False,1,Tuesday,23
588098,588098,1038442,ad,False,3,Tuesday,23
588099,588099,1496395,ad,False,1,Tuesday,23
588100,588100,1237779,ad,False,1,Tuesday,23


In [5]:
# una vez que tengamos nuestro DataFrame preparado con todas las columnas que queremos vamos a crear una función que no haga una exploración inicial del conjunto de datos
def exploracion_dataframe(dataframe, columna_control):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas, agrupadas por la columna de control.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.
    - columna_control (str): El nombre de la columna que se utilizará como control para dividir el DataFrame.

    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Los duplicados que tenemos en el conjunto de datos son: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    
    # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()).head())    
    
    # como estamos en un problema de A/B testing y lo que realmente nos importa es comparar entre el grupo de control y el de test, los principales estadísticos los vamos a sacar de cada una de las categorías
    
    for categoria in dataframe[columna_control].unique():
        
        dataframe_filtrado = dataframe[dataframe[columna_control] == categoria]
    
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas categóricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe(include = "O").T)
        
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas numéricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe().T)


In [6]:
exploracion_dataframe(df,"test group")

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos



 ..................... 

Los tipos de las columnas son:


,tipo_dato
Unnamed: 0,int64
user id,int64
test group,object
converted,bool
total ads,int64
most ads day,object
most ads hour,int64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna TEST GROUP tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,564577.0,2.939145e+05,168039.202368,0.0,149003.0,294089.0,438890.0,588100.0
user id,564577.0,1.327314e+06,188918.061566,1000000.0,1163686.0,1327362.0,1490914.0,1654483.0
total ads,564577.0,2.482337e+01,43.750456,1.0,4.0,13.0,27.0,2065.0
most ads hour,564577.0,1.447590e+01,4.841808,0.0,11.0,14.0,18.0,23.0



 ..................... 

Los principales estadísticos de las columnas categóricas para el PSA son: 


,count,unique,top,freq
test group,23524,1,psa,23524
most ads day,23524,7,Thursday,3905



 ..................... 

Los principales estadísticos de las columnas numéricas para el PSA son: 


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,23524.0,297302.795018,206995.969959,18.0,83618.25,293321.5,526178.75,588081.0
user id,23524.0,911761.500000,6790.938202,900000.0,905880.75,911761.5,917642.25,923523.0
total ads,23524.0,24.761138,42.860720,1.0,4.00,12.0,26.00,907.0
most ads hour,23524.0,14.304923,4.656239,0.0,11.00,14.0,18.00,23.0


In [7]:
# Otra Forma de hacerlo

# Dividir los grupos A y B
group_A = df[df["test group"] == "ad"]  #  Grupo de control
group_B = df[df["test group"] == "psa"]

In [8]:
# Suponiendo que 'df' es tu DataFrame
tipos_de_columnas = df.dtypes

# Imprimir los tipos de columnas
print("Tipos de las columnas:")
print(tipos_de_columnas)

Tipos de las columnas:
Unnamed: 0        int64
user id           int64
test group       object
converted          bool
total ads         int64
most ads day     object
most ads hour     int64
dtype: object


In [9]:
def investigar_valores_nulos(dataframe, group_name):

    # Filtrar el DataFrame por el grupo especificado
    group = dataframe[dataframe["test group"] == group_name]
    
    # Contar los valores nulos en cada columna del grupo
    return group.isnull().sum()



In [10]:
# Llamar a la función para investigar valores nulos en el grupo "ad"
valores_nulos_ad = investigar_valores_nulos(df, "ad")
print(f"Valores nulos en el grupo 'ad':\n{valores_nulos_ad}")

# Llamar a la función para investigar valores nulos en el grupo "psa"
valores_nulos_psa = investigar_valores_nulos(df, "psa")
print(f"Valores nulos en el grupo 'psa':\n{valores_nulos_psa}")


Valores nulos en el grupo 'ad':
Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64
Valores nulos en el grupo 'psa':
Unnamed: 0       0
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64


In [11]:
def eliminar_duplicados_en_grupos(dataframe, grupo_columna):

    # Inicializar un DataFrame vacío para almacenar los resultados
    df_sin_duplicados = pd.DataFrame()
    
    # Obtener los grupos únicos
    grupos = dataframe[grupo_columna].unique()
    
    # Iterar sobre cada grupo, eliminar duplicados y concatenar los resultados
    for grupo in grupos:
        grupo_actual = dataframe[dataframe[grupo_columna] == grupo]
        grupo_sin_duplicados = grupo_actual.drop_duplicates()
        df_sin_duplicados = pd.concat([df_sin_duplicados, grupo_sin_duplicados], ignore_index=True)
    
    return df_sin_duplicados



In [12]:
# Llamar a la función para eliminar duplicados en cada grupo
df_sin_duplicados = eliminar_duplicados_en_grupos(df, "test group")

# Imprimir el DataFrame resultante sin duplicados
print("DataFrame sin duplicados en cada grupo:")
print(df_sin_duplicados)


DataFrame sin duplicados en cada grupo:
        Unnamed: 0  user id test group  converted  total ads most ads day  \
0                0  1069124         ad      False        130       Monday   
1                1  1119715         ad      False         93      Tuesday   
2                2  1144181         ad      False         21      Tuesday   
3                3  1435133         ad      False        355      Tuesday   
4                4  1015700         ad      False        276       Friday   
...            ...      ...        ...        ...        ...          ...   
588096      588052   900959        psa      False         16      Tuesday   
588097      588063   902828        psa      False          3      Tuesday   
588098      588066   914578        psa      False          1      Tuesday   
588099      588069   909042        psa      False          6      Tuesday   
588100      588081   904721        psa      False         16      Tuesday   

        most ads hour  
0          

In [13]:
def valores_columnas_categoricas(dataframe, grupo_columna, grupo_A_nombre, grupo_B_nombre):

    valores_categoricos = {}
    
    # Función para obtener valores únicos y su conteo
    def obtener_valores_y_conteo(grupo):
        valores_unicos = grupo[columna].value_counts().index.tolist()
        conteo = grupo[columna].value_counts().tolist()
        return valores_unicos, conteo
    
    # Filtrar y extraer valores únicos y conteo de columnas categóricas para grupo A
    grupo_A = dataframe[dataframe[grupo_columna] == grupo_A_nombre]
    for columna in grupo_A.select_dtypes(include=['object']):
        valores_categoricos[f"{grupo_A_nombre}_{columna}"] = obtener_valores_y_conteo(grupo_A)
    
    # Filtrar y extraer valores únicos y conteo de columnas categóricas para grupo B
    grupo_B = dataframe[dataframe[grupo_columna] == grupo_B_nombre]
    for columna in grupo_B.select_dtypes(include=['object']):
        valores_categoricos[f"{grupo_B_nombre}_{columna}"] = obtener_valores_y_conteo(grupo_B)
    
    return valores_categoricos





In [14]:
columnas_categoricas = valores_columnas_categoricas(df, "test group", "ad", "psa")

# Imprimir los valores únicos y el conteo de columnas categóricas
print("Valores únicos y conteo de columnas categóricas:")
for columna, (valores, conteo) in columnas_categoricas.items():
    print(f"{columna}:")
    for valor, count in zip(valores, conteo):
        print(f"   {valor}: {count}")

Valores únicos y conteo de columnas categóricas:
ad_test group:
   ad: 564577
ad_most ads day:
   Friday: 88805
   Monday: 83571
   Sunday: 82332
   Thursday: 79077
   Saturday: 78802
   Wednesday: 77418
   Tuesday: 74572
psa_test group:
   psa: 23524
psa_most ads day:
   Thursday: 3905
   Friday: 3803
   Monday: 3502
   Wednesday: 3490
   Sunday: 3059
   Tuesday: 2907
   Saturday: 2858


In [15]:
def estadisticos_columnas_numericas_por_grupo(dataframe, grupo_columna, grupos):
    
    estadisticos_por_grupo = {}
    
    for grupo_nombre in grupos:
        # Filtrar el DataFrame por el grupo especificado
        grupo = dataframe[dataframe[grupo_columna] == grupo_nombre]
        
        # Obtener estadísticos descriptivos de las columnas numéricas
        estadisticos = grupo.describe()
        
        # Transponer el DataFrame de estadísticos para una mejor visualización
        estadisticos_transpuesto = estadisticos.T
        
        # Agregar estadísticos al diccionario
        estadisticos_por_grupo[grupo_nombre] = estadisticos_transpuesto
    
    return estadisticos_por_grupo



In [16]:

# Definir la lista de grupos
grupos = ["ad", "psa"]

# Llamar a la función para obtener estadísticos de columnas numéricas para cada grupo
estadisticos_por_grupo = estadisticos_columnas_numericas_por_grupo(df, "test group", grupos)

# Imprimir los estadísticos para cada grupo
for grupo, estadisticos in estadisticos_por_grupo.items():
    print(f"Principales estadísticos de las columnas numéricas para el grupo {grupo}:")
    print(estadisticos)
    print()  # Para separar los resultados de cada grupo

Principales estadísticos de las columnas numéricas para el grupo ad:
                  count          mean            std        min        25%  \
Unnamed: 0     564577.0  2.939145e+05  168039.202368        0.0   149003.0   
user id        564577.0  1.327314e+06  188918.061566  1000000.0  1163686.0   
total ads      564577.0  2.482337e+01      43.750456        1.0        4.0   
most ads hour  564577.0  1.447590e+01       4.841808        0.0       11.0   

                     50%        75%        max  
Unnamed: 0      294089.0   438890.0   588100.0  
user id        1327362.0  1490914.0  1654483.0  
total ads           13.0       27.0     2065.0  
most ads hour       14.0       18.0       23.0  

Principales estadísticos de las columnas numéricas para el grupo psa:
                 count           mean            std       min        25%  \
Unnamed: 0     23524.0  297302.795018  206995.969959      18.0   83618.25   
user id        23524.0  911761.500000    6790.938202  900000.0  905880

In [17]:
#Planteamiento de las hipótesis: 

#Hipótesis nula (H0): No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

#Hipótesis alternativa (H1): Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".z

In [18]:
#Calculo de la tasa de conversion:
# Calcular la tasa de conversión para el grupo A ("ad")
converted_A = group_A["converted"].sum()
total_users_A = len(group_A)
conversion_rate_A = converted_A / total_users_A

print(f"Tasa de conversión para el grupo 'ad': {conversion_rate_A:.2%}")

# Calcular la tasa de conversión para el grupo B ("psa")
converted_B = group_B["converted"].sum()
total_users_B = len(group_B)
conversion_rate_B = converted_B / total_users_B

print(f"Tasa de conversión para el grupo 'psa': {conversion_rate_B:.2%}")

Tasa de conversión para el grupo 'ad': 2.55%
Tasa de conversión para el grupo 'psa': 1.79%


In [19]:

# Realizar la prueba t de Student para muestras independientes
t_statistic, p_value = stats.ttest_ind(group_A["converted"], group_B["converted"], equal_var=True)

# Imprimir los resultados de la prueba de hipótesis
print("\nPrueba de hipótesis utilizando la prueba t de Student:")
print(f"Estadístico t: {t_statistic}")
print(f"Valor p: {p_value}")

# Interpretación del valor p
alpha = 0.05  # Nivel de significancia
if p_value < alpha:
    print("Conclusión: Se rechaza la hipótesis nula (H0). Existe una diferencia significativa en las tasas de conversión entre los grupos 'ad' y 'psa'.")
else:
    print("Conclusión: No se puede rechazar la hipótesis nula (H1). No hay suficiente evidencia para concluir que existe una diferencia significativa en las tasas de conversión entre los grupos 'ad' y 'psa'.")



Prueba de hipótesis utilizando la prueba t de Student:
Estadístico t: 7.37040597428566
Valor p: 1.7033052627831264e-13
Conclusión: Se rechaza la hipótesis nula (H0). Existe una diferencia significativa en las tasas de conversión entre los grupos 'ad' y 'psa'.


In [33]:


# tabla de contigencia
contingency_table = pd.crosstab(df['test group'], df['converted'])

# test de p-value
chi2, p, dof, expected = chi2_contingency(contingency_table)

chi2, p, dof, expected


(54.005823883685245,
 1.9989623063390075e-13,
 1,
 array([[550327.71899045,  14249.28100955],
        [ 22930.28100955,    593.71899045]]))

In [31]:
# Imprimir el resultado de la prueba
alpha = 0.05
if p_value < alpha:
    print("Hay una diferencia significativa en las tasas de clics entre el Grupo de Control y el Grupo de Prueba.")
    print("\n ---------- \n")
    print("""
          Los resultados sugieren que existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, nuestro nuevo sistema tiene los efectos deseados y deberíamos cambiar la nueva versión de anuncios   
          """)
else:
    print("No hay evidencia de una diferencia significativa en las tasas de clics entre los grupos.")
    print("\n ---------- \n")
    print(""" 
          Los resultados sugieren que no existe evidencia estadística para afirmar que las medias de las muestras son distintas,
          por lo que la nueva campaña no esta ayudando a nuestro problema. 
          """)

Hay una diferencia significativa en las tasas de clics entre el Grupo de Control y el Grupo de Prueba.

 ---------- 


          Los resultados sugieren que existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, nuestro nuevo sistema tiene los efectos deseados y deberíamos cambiar la nueva versión de anuncios   
          


## Prueba Z
- **Hipótesis Nula (H0):** No hay diferencia significativa  entre el grupo de control y el grupo de prueba.

- **Hipótesis Alternativa (H1):** Hay una diferencia significativa entre el grupo de control y el grupo de prueba.



In [21]:

# Calcular el total de registros y conversiones para cada grupo
ad_total = df[df['test group'] == 'ad'].shape[0]
ad_converted = df[(df['test group'] == 'ad') & (df['converted'] == True)].shape[0]
psa_total = df[df['test group'] == 'psa'].shape[0]
psa_converted = df[(df['test group'] == 'psa') & (df['converted'] == True)].shape[0]

# Proporciones de conversión
p1 = ad_converted / ad_total
p2 = psa_converted / psa_total

# Proporción combinada
p_combined = (ad_converted + psa_converted) / (ad_total + psa_total)

# Valor Z
z = (p1 - p2) / np.sqrt(p_combined * (1 - p_combined) * (1 / ad_total + 1 / psa_total))

# Valor p a partir del valor Z
p_value = 2 * (1 - norm.cdf(abs(z)))

p1, p2, p_combined, z, p_value


(0.025546559636683747,
 0.01785410644448223,
 0.02523886203220195,
 7.3700781265454145,
 1.7053025658242404e-13)

In [22]:
#Si `p_value < 0.05`: Rechazamos la hipótesis nula. Hay evidencia suficiente para decir que hay una diferencia significativa entre las dos muestras.

In [ ]:
#Si hay una diferencia entre el ad y psa con lo cual a primera vista y teniendo en cuenta los datos de conversion se demuestra que la ad es mejor.

In [ ]:
#Ejercicio 2 Objetivo del experimento: Comparar la cantidad promedio de anuncios vistos por usuario entre los grupos de prueba "ad" y "psa" para determinar si el tipo de anuncio afecta la cantidad de anuncios que los usuarios ven

In [35]:
# Calcular la media de la cantidad de anuncios vistos por usuario en el grupo 'ad'
media_ad = df[df['test group'] == 'ad']['total ads'].mean()

# Calcular la media de la cantidad de anuncios vistos por usuario en el grupo 'psa'
media_psa = df[df['test group'] == 'psa']['total ads'].mean()

print(f"El promedio de anuncios vistos por usuario en el grupo A es: {media_ad}")
print(f"El promedio de anuncios vistos por usuario en el grupo B es: {media_psa}")

El promedio de anuncios vistos por usuario en el grupo A es: 24.823365103431417
El promedio de anuncios vistos por usuario en el grupo B es: 24.761137561639178


In [ ]:
#Planteamiento de las hipótesis: 

#Hipótesis nula (H0): No hay diferencia significativa en los anuncios  vistos para ads y psa".

#Hipótesis alternativa (H1): Existe una diferencia significativa en los anuncios  vistos para ads y psa".

In [37]:
# Realizar la prueba t de Student para muestras independientes
t_statistic, p_value = ttest_ind(df[df['test group'] == 'ad']['total ads'], 
                                 df[df['test group'] == 'psa']['total ads'])

# Nivel de significancia
alpha = 0.05

# Imprimir los resultados
print(f"Media de anuncios vistos en 'ad': {media_ad}")
print(f"Media de anuncios vistos en 'psa': {media_psa}")
print(f"Valor p del test t de Student: {p_value}")

# Interpretación de los resultados
if p_value < alpha:
    print("\nHay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos 'ad' y 'psa'.")
else:
    print("\nNo hay evidencia suficiente para afirmar que hay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos.")

Media de anuncios vistos en 'ad': 24.823365103431417
Media de anuncios vistos en 'psa': 24.761137561639178
Valor p del test t de Student: 0.8306134186810267

No hay evidencia suficiente para afirmar que hay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos.


In [38]:
from scipy.stats import mannwhitneyu



# Obtener las cantidades de anuncios vistos para cada grupo
cantidad_anuncios_ad = df[df['test group'] == 'ad']['total ads']
cantidad_anuncios_psa = df[df['test group'] == 'psa']['total ads']

# Realizar la prueba de Wilcoxon-Mann-Whitney
statistic, p_value = mannwhitneyu(cantidad_anuncios_ad, cantidad_anuncios_psa, alternative='two-sided')

# Nivel de significancia
alpha = 0.05

# Imprimir los resultados
print(f"Prueba de Wilcoxon-Mann-Whitney:")
print(f"Estadístico de prueba: {statistic}")
print(f"Valor p: {p_value}")

# Interpretación de los resultados
if p_value < alpha:
    print("\nHay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos 'ad' y 'psa'.")
else:
    print("\nNo hay evidencia suficiente para afirmar que hay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos.")


Prueba de Wilcoxon-Mann-Whitney:
Estadístico de prueba: 6808288222.0
Valor p: 4.6909108547205694e-11

Hay una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos 'ad' y 'psa'.


In [ ]:
#La prueba t de Student puede no ser adecuada si los datos no cumplen con los supuestos de normalidad, mientras que la U de Mann-Whitney es más robusta en tales casos y puede detectar diferencias basadas en la ubicación de las distribuciones, incluso si las diferencias en las medias no son significativas.
#  la prueba de Wilcoxon-Mann-Whitney es más robusta frente a datos que no cumplen con la normalidad
#El valor p (4.6909108547205694e-11, que es muy pequeño) es la probabilidad de observar una diferencia tan grande en los rangos entre los grupos si la hipótesis nula (que no hay diferencia entre los grupos) fuera cierta. Un valor p pequeño sugiere evidencia fuerte en contra de la hipótesis nula.

 La prueba t de Student asume normalidad en los datos. Si los datos cumplen con este supuesto, la prueba t puede ser más poderosa para detectar diferencias en las medias. Sin embargo, si los datos no son normales  como parece ser en nuestro caso la prueba t puede no ser apropiada y podría dar un valor p más alto, indicando que no hay diferencia significativa.
La prueba t se centra en las diferencias de medias, mientras que la prueba de Wilcoxon-Mann-Whitney se centra en las diferencias de la mediana. Es posible que las medias sean similares (como indicado por el valor p alto en la prueba t), pero que haya diferencias en la ubicación de las distribuciones que son detectadas por la prueba de Wilcoxon-Mann-Whitney.

El valor p extremadamente pequeño (4.6909108547205694e-11) indica que es altamente improbable que la diferencia en la cantidad de anuncios vistos entre los grupos 'ad' y 'psa' sea debida al azar, bajo la suposición de que no hay diferencia real entre los grupos.

Por lo tanto, con este valor p tan pequeño, se rechaza la hipótesis nula de que no hay diferencia en la cantidad de anuncios vistos entre los grupos 'ad' y 'psa'. Esto sugiere que hay una diferencia significativa en la cantidad de anuncios vistos por usuario entre los dos grupos.

Enla prueba de Wilcoxon-Mann-Whitney  proporciona una sólida de que existe una diferencia significativa en la cantidad de anuncios vistos por usuario entre los grupos 'ad' y 'psa', sin depender de la normalidad de los datos, 

En este caso específico, dado el valor p alto del test t de Student (0.8306), no podemos afirmar que haya una diferencia significativa en la cantidad promedio de anuncios vistos entre los grupos 'ad' y 'psa'. Sin embargo, es importante considerar que la prueba de Wilcoxon-Mann-Whitney mostró una diferencia significativa en términos de la ubicación de las distribuciones de los datos. Esto resalta la importancia de elegir la prueba estadística adecuada según la naturaleza de tus datos y los supuestos subyacentes.